# Model Summary Clavicle CT

In [1]:
from clavicle_ct.net.agenet import agenet18_3d
from clavicle_ct.litmodel import LitClavicle
from clavicle_ct.data import ClavicleDataModule
from torchinfo import summary

batch_size = 8
annotation_file_train = "/data_fae_uq/clavicle_ct/annotations_train.csv"
img_train_base_dir = "/data_fae_uq/clavicle_ct/preprocessed/"

datamodule = ClavicleDataModule(
    annotation_file_train=annotation_file_train,
    annotation_file_val=None,
    annotation_file_test=None,
    img_train_base_dir=img_train_base_dir,
    batch_size=batch_size,
    with_sex_input=True,
    num_workers=1,
)
datamodule.setup("train")
x, y = next(iter(datamodule.train_dataloader()))


/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


### High Level Summary

Top-level Summary and first layers of `AgeNet`

In [2]:
output_neurons = 1  # not a variance net
net = agenet18_3d(use_dropout=True, use_sex=True, num_classes=output_neurons)
model = LitClavicle(net=net)

summary(
    model,
    input_data=[x],
    col_names=["input_size", "output_size", "num_params"],
    depth=2,
    col_width=25,
    row_settings=["ascii_only"],
)


Layer (type)                             Input Shape               Output Shape              Param #
LitClavicle                              [8, 1, 112, 112, 112]     [8, 1]                    --
+ AgeNet_3D                              [8, 1, 112, 112, 112]     [8, 1]                    --
|    + Conv3d                            [8, 1, 112, 112, 112]     [8, 64, 56, 56, 56]       8,064
|    + BatchNorm3d                       [8, 64, 56, 56, 56]       [8, 64, 56, 56, 56]       128
|    + ReLU                              [8, 64, 56, 56, 56]       [8, 64, 56, 56, 56]       --
|    + MaxPool3d                         [8, 64, 56, 56, 56]       [8, 64, 28, 28, 28]       --
|    + Sequential                        [8, 64, 28, 28, 28]       [8, 64, 28, 28, 28]       442,880
|    + Sequential                        [8, 64, 28, 28, 28]       [8, 128, 14, 14, 14]      1,557,760
|    + Sequential                        [8, 128, 14, 14, 14]      [8, 256, 7, 7, 7]         6,228,480
|    + Seque

### Detailed Architecture of Sequential Block (Building Block Adapted From ResNet)

In [3]:
summary(
    model.net.layer1,
    input_size=(batch_size, 64, 28, 28, 28),
    col_names=["input_size", "output_size", "kernel_size"],
    depth=3,
    col_width=25,
    row_settings=["ascii_only"]
)

Layer (type)                             Input Shape               Output Shape              Kernel Shape
Sequential                               [8, 64, 28, 28, 28]       [8, 64, 28, 28, 28]       --
+ BuildingBlock_3D                       [8, 64, 28, 28, 28]       [8, 64, 28, 28, 28]       --
|    + Conv3d                            [8, 64, 28, 28, 28]       [8, 64, 28, 28, 28]       [3, 3, 3]
|    + BatchNorm3d                       [8, 64, 28, 28, 28]       [8, 64, 28, 28, 28]       --
|    + ReLU                              [8, 64, 28, 28, 28]       [8, 64, 28, 28, 28]       --
|    + Conv3d                            [8, 64, 28, 28, 28]       [8, 64, 28, 28, 28]       [3, 3, 3]
|    + BatchNorm3d                       [8, 64, 28, 28, 28]       [8, 64, 28, 28, 28]       --
|    + ReLU                              [8, 64, 28, 28, 28]       [8, 64, 28, 28, 28]       --
+ BuildingBlock_3D                       [8, 64, 28, 28, 28]       [8, 64, 28, 28, 28]       --
|    + Conv3d   